In [52]:
import matplotlib.pyplot as plt

from mne.datasets import sample
from mne.io import  RawArray

from mne_realtime import LSLClient, MockLSLStream, RtEpochs, MockRtClient
import numpy as np
import pandas as pd
import mne as mne



In [53]:
# Load a file to stream raw data
df = pd.read_csv('data/ssvep-20trials-3s-chaky-bigsquare.csv')
df.rename(columns={'Unnamed: 1':'O1',
                          'Unnamed: 2':'Oz',
                          'Unnamed: 3':'O2'
                      }, 
                 inplace=True)
df = df.drop(["timestamps", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8"], axis=1)
print(df)


              O1         Oz         O2  Marker
0      38956.250  27233.611  44154.887       0
1      39004.645  27278.182  44197.668       0
2      39005.805  27278.383  44199.301       0
3      38978.270  27251.068  44177.395       0
4      38944.137  27219.082  44149.145       0
...          ...        ...        ...     ...
62431  45353.141  34611.250  52799.133       0
62432  45371.000  34628.844  52808.699       0
62433  45391.812  34651.441  52827.902       0
62434  45399.434  34662.551  52838.652       0
62435  45393.398  34660.203  52834.293       0

[62436 rows x 4 columns]


In [54]:
# convert dataframe to raw array
data = df.to_numpy()
data = data.transpose()
print(data)

[[38956.25  39004.645 39005.805 ... 45391.812 45399.434 45393.398]
 [27233.611 27278.182 27278.383 ... 34651.441 34662.551 34660.203]
 [44154.887 44197.668 44199.301 ... 52827.902 52838.652 52834.293]
 [    0.        0.        0.    ...     0.        0.        0.   ]]


In [55]:
sfreq = 250
#Channel name
ch_names = ['O1', 'Oz', 'O2','Marker']
ch_types = ['eeg','eeg','eeg','misc']
# Create the info structure needed by MNE
info = mne.create_info(ch_names, sfreq,ch_types=ch_types)

# Finally, create the Raw object
raw = mne.io.RawArray(data, info)
# print (raw.times)

Creating RawArray with float64 data, n_channels=4, n_times=62436
    Range : 0 ... 62435 =      0.000 ...   249.740 secs
Ready.


In [56]:
host = 'mne_stream'
stream = MockLSLStream(host, raw, 'eeg')

epoch = ''
# Let's observe it
# plt.ion()  # make plot interactive
# _, ax = plt.subplots(1)
    
with LSLClient(info=raw.info, host=host, wait_max=3) as client:
    stream.start()
    client_info = client.get_measurement_info()
    print(client_info)
#     sfreq = int(client_info['sfreq'])
# #     print(client_info)


#     # let's observe ten seconds of data
    for ii in range(2):
#         plt.cla()
        epoch = client.get_data_as_epoch(n_samples=sfreq)
        print(epoch.getData())
#         epoch.average().plot(axes=ax)
#         plt.pause(1)
    
# Let's terminate the mock LSL stream
stream.stop()

Client: Waiting for server to start
Client: Connected
now sending data...
<Info | 7 non-empty values
 bads: []
 ch_names: O1, Oz, O2
 chs: 3 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 3
 projs: []
 sfreq: 250.0 Hz
>
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Stopping stream...


TypeError: __init__() missing 1 required positional argument: 'tmax'